In [38]:
file_path = "/user/hive/warehouse/bronze/olist_geolocation"

df = (
    spark.read
    .format("parquet")
    .load(file_path)
)

df2 = df.limit(200)

In [40]:
from geopy.geocoders import Nominatim
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf, lit, concat
import time

In [41]:
@udf()
def find_city(lat, long):
    geolocator = Nominatim(user_agent="MBA_FIAP")
    try:
        latlong = str(lat) + ", " + str(long)
        location = geolocator.reverse(latlong)
        return location.raw["address"]['postcode']
    except:
        return 'No city'

df2 = df2.withColumn('coord', find_city(df2.geolocation_lat, df2.geolocation_lng))

In [10]:
df2.printSchema()

root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)
 |-- coord: string (nullable = true)



In [42]:
start = time.time()
df2.show()
end = time.time()
print(end-start)

+---------------------------+-------------------+-------------------+----------------+-----------------+----------+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|     coord|
+---------------------------+-------------------+-------------------+----------------+-----------------+----------+
|                      29107| -20.34584830411424| -40.29461716546357|      vila velha|               ES|29.100-200|
|                      29177|-20.122435947833026| -40.31909381793521|           serra|               ES| 29177-129|
|                      29105|-20.368295871918747| -40.31092289293257|      vila velha|               ES| 29105-550|
|                      29173| -20.19652746890127|-40.198275466345144|           serra|               ES| 29173-039|
|                      29109|-20.355711075892653| -40.31492752529756|      vila velha|               ES| 29109-213|
|                      29132| -20.39209755581655| -40.47152277013746|   

In [ ]:
# Pego a latitude e a longitude do df e com isso busco o cep e o endereço
dataCollect = df2.collect()
for row in dataCollect:
    v_lat_lon = (str(row['geolocation_lat']) + ", " + str(row['geolocation_lng']))
    print(v_lat_lon)

    geolocator = Nominatim(user_agent="MBA_FIAP")
    location = geolocator.reverse(v_lat_lon)
    cep = (location.raw['address']['postcode'])

    if len(cep) < 8:
        cep = cep.zfill(8)
        cep = '{}-{}'.format(cep[:5], cep[5:8])
    print(cep) 

    address = get_address_from_cep(cep, webservice=WebService.VIACEP)

    print(address['logradouro'])

    print('-----------')

###### É preciso ver o que fazer quando o o retorno da geopy não for CEP
### Crio novo df limitando a 10 rows e só do RJ pra testar
df2 = df.filter(df.geolocation_state=='RJ').limit(10)